# Electrical Power Generation 2

## Objective and Prerequisites

This model is an example of an electrical power generation problem (also known as a unit commitment problem). It selects an optimal set of power stations to turn on in order to satisfy anticipated power demand over a 24-hour time horizon.  It is an extension of an earlier 'Electrical Power Generation' example (example 15 from HP Williams); the extension adds the option of using hydroelectric power plants to satisfy demand.

This model is example 16 from the fifth edition of Model Building in Mathematical Programming, by H. Paul Williams on pages 271-272 and 326-327.

This example is of intermediate difficulty; we assume that you know Python and the Gurobi Python API and that you have some knowledge of building mathematical optimization models.

**Note:** You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license](https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=Github&utm_medium=website_JupyterME&utm_campaign=CommercialDataScience) as a *commercial user*, or download a [free license](https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=Github&utm_medium=website_JupyterME&utm_campaign=AcademicDataScience) as an *academic user*.

---
## Problem Description

In this problem, thermal power generation units are grouped into three distinct types, with different characteristics for each type (power output, cost per megawatt hour, startup cost, etc.).  A unit can be on or off, with a startup cost associated with transitioning from off to on, and power output that can fall anywhere between a specified minimum and maximum value when the unit is on.  There are also two hydroelectric plants available, also with different cost and power generation characteristics.  A 24-hour time horizon is divided into 5 discrete time periods, each with an expected total power demand.  The model decides which units to turn on, and when, in order to satisfy demand for each time period.  The model also captures a reserve requirement, where the selected power plants must be capable of increasing their output, while still respecting their maximum output, in order to cope with the situation where actual demand exceeds predicted demand.

A set of generators is available to satisfy power demand for the following day.  Anticipated demand is as follows:

| Time Period | Demand (megawatts) |
| --- | --- |
| 12 pm to 6 am | 15000 |
| 6 am to 9 am | 30000 |
| 9 am to 3 pm | 25000 |
| 3 pm to 6 pm | 40000 |
| 6 pm to 12 pm | 27000 |

Thermal generators are grouped into three types, with the following minimum and maximum output for each type (when they are on):

| Type | Number available | Minimum output (MW) | Maximum output (MW) |
| --- | --- | --- | --- |
| 0 | 12 |  850 | 2000 |
| 1 | 10 | 1250 | 1750 |
| 2 | 5 | 1500 | 4000 |

There are costs associated with using a thermal generator: a cost per hour when the generator is on (and generating its minimum output), a cost per megawatt hour above its minimum, and a startup cost for turning a generator on:

| Type | Cost per hour (when on) | Cost per MWh above minimum | Startup cost |
| --- | --- | --- | --- |
| 0 | $\$1000$ | $\$2.00$ | $\$2000$ |
| 1 | $\$2600$ | $\$1.30$ | $\$1000$ |
| 2 | $\$3000$ | $\$3.00$ | $\$500$ |

Two hydroelectric generators are also available, each with a fixed power output (when on):

| Hydro plant | Output (MW) |
| --- | --- |
| A | 900 |
| B | 1400 |

The costs associated with using a hydro plant are slightly different.  There's an hourly cost, but it is much smaller than the hourly cost of a thermal generator.  The real cost for a hydroelectric plant comes from depletion of the water in the reservoir, which happens at different rates for the two units.  The reservoir must be replenished before the end of the time horizon by pumping water into it, which consumes electricity.  A hydroelectric plant also has a startup cost.

| Hydro plant | Cost per hour (when on) | Startup cost | Reservoir depth reduction (m/hr) |
| --- | --- | --- | --- |
| A | $\$90$ | $\$1500$ | 0.31 |
| B | $\$150$ | $\$1200$ | 0.47 |

Pumping water into the reservoir consumes electricity at a rate of 3000 MWh of electricity per meter of height.  The height of the reservoir at the end of the time horizon must be equal to the height at the beginning.

Generators must meet predicted demand, but they must also have sufficient reserve capacity to be able to cope with the situation where actual demand exceeds predicted demand.  For this model, the set of selected thermal generators plus the set of hydro generators must be able to produce as much as 115% of predicted demand.

Which generators should be committed to meet anticipated demand in order to minimize total cost?

---
## Model Formulation

### Sets and Indices

$t \in \text{Types}=\{0,1,2\}$: Types of thermal generators.

$h \in \text{HydroUnits}=\{0,1\}$: Two hydro generators.

$p \in \text{Periods}=\{0,1,2,3,4\}$: Time periods.

### Parameters

$\text{period_hours}_{p} \in \mathbb{N}^+$: Number of hours in each time period.

$\text{demand}_p \in \mathbb{R}^+$: Total power demand for time period $p$.

$\text{generators}_t \in \mathbb{N}^+$: Number of thermal generators of type $t$.

$\text{start0} \in \mathbb{N}^+$: Number of thermal generators that are on at the beginning of the time horizon (and available in time period 0 without paying a startup cost).

$\text{min_output}_t \in \mathbb{R}^+$: Minimum output for thermal generator type $t$ (when on).

$\text{max_output}_t \in \mathbb{R}^+$: Maximum output for thermal generator type $t$.

$\text{base_cost}_t \in \mathbb{R}^+$: Minimum operating cost (per hour) for a thermal generator of type $t$.

$\text{per_mw_cost}_t \in \mathbb{R}^+$: Cost to generate one additional MW (per hour) for a thermal generator of type $t$.

$\text{startup_cost}_t \in \mathbb{R}^+$: Startup cost for thermal generator of type $t$.

$\text{hydro_load}_h \in \mathbb{R}^+$: Output for hydro generator $h$.

$\text{hydro_cost}_h \in \mathbb{R}^+$: Cost for operating hydro generator $h$.

$\text{hydro_startup_cost}_h \in \mathbb{R}^+$: Startup cost for hydro generator $h$.

$\text{hydro_height_reduction}_h \in \mathbb{R}^+$: Hourly reduction in reservoir height from operating hydro generator $h$.

### Decision Variables

$\text{ngen}_{t,p} \in \mathbb{N}^+$: Number of thermal generators of type $t$ that are on in time period $p$.

$\text{output}_{t,p} \in \mathbb{R}^+$: Total power output from thermal generators of type $t$ in time period $p$.

$\text{nstart}_{t,p} \in \mathbb{N}^+$: Number of thermal generators of type $t$ to start in time period $p$.

$\text{hydro}_{h,p} \in [0,1]$: Indicates whether hydro generators $h$ is on in time period $p$.

$\text{hydro_start}_{h,p} \in [0,1]$: Indicates whether hydro generator $h$ starts in time period $p$.

$\text{height}_{p} \in \mathbb{R}^+$: Height of reservoir in time period $p$.

$\text{pumping}_{p} \in \mathbb{R}^+$: Power used to replenish reservoir in time period $p$.


### Objective Function

- **Cost**: Minimize the cost (in USD) to satisfy the predicted electricity demand.

\begin{equation}
\text{Minimize} \quad Z_{on} + Z_{extra} + Z_{startup} + Z_{hydro} + Z_{hydro\_startup}
\end{equation}

\begin{equation}
Z_{on} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{base_cost}_t*\text{ngen}_{t,p}}
\end{equation}
\begin{equation}
Z_{extra} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{per_mw_cost}_t*(\text{output}_{t,p} - \text{min_load}_t})
\end{equation}
\begin{equation}
Z_{startup} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{startup_cost}_t*\text{nstart}_{t,p}}
\end{equation}
\begin{equation}
Z_{hydro} = \sum_{(h,p) \in \text{HydroUnits} \times \text{Periods}}{\text{hydro_cost}_h*\text{hydro}_{h,p}}
\end{equation}
\begin{equation}
Z_{hydro\_startup} = \sum_{(h,p) \in \text{HydroUnits} \times \text{Periods}}{\text{hydro_startup_cost}_h*\text{hydro_start}_{h,p}}
\end{equation}


### Constraints

- **Available generators**: Number of generators used must be less than or equal to the number available.

\begin{equation}
\text{ngen}_{t,p} \leq \text{generators}_t  \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

- **Demand**: Total power generated across all generator types must meet anticipated demand plus pumping for each time period $p$.

\begin{equation}
\sum_{t \in \text{Types}}{\text{output}_{t,p}} +
\sum_{h \in \text{HydroUnits}}{\text{hydro_load}_h*\text{hydro}_{h,p}} \geq
\text{demand}_p + \text{pumping}_p \quad \forall p \in \text{Periods}
\end{equation}

- **Min/max generation**: Power generation must respect thermal generator min/max values.

\begin{equation}
\text{output}_{t,p} \geq \text{min_output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

\begin{equation}
\text{output}_{t,p} \leq \text{max_output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

- **Reserve**: Selected thermal generators plus hydro units must be able to satisfy demand that is as much as 15% above the prediction.

\begin{equation}
\sum_{t \in \text{Types}}{\text{max_output}_t*\text{ngen}_{t,p}} +
\sum_{h \in \text{HydroUnits}}{\text{hydro_load}_h} \geq 1.15 * \text{demand}_p \quad \forall p \in \text{Periods}
\end{equation}

- **Thermal startup**: Establish relationship between number of active thermal generators and number of startups (use $start0$ for period before the time horizon starts)

\begin{equation}
\text{ngen}_{t,p} \leq \text{ngen}_{t,p-1} + \text{nstart}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

- **Hydro startup**: Establish relationship between hydro generator state and number of hydro startups (assume hydro plants are off at the beginning of the horizon)

\begin{equation}
\text{hydro}_{h,p} \leq \text{hydro}_{h,p-1} + \text{hydro_start}_{h,p} \quad \forall (h,p) \in \text{HydroUnits} \times \text{Periods}
\end{equation}

- **Reservoir height**: Track reservoir height.   Note that the height at the end of the final time period must equal the height at the beginning of the first.

- Reservoir level constraints: Height increases due to pumping activity and decreases due to hydroelectric generation.

\begin{equation}
\text{height}_{p} = \text{height}_{p-1}  + \text{period_hours}_{p}*\text{pumping}_{p}/3000 -
\sum_{h \in \text{HydroUnits}}{\text{period_hours}_{p}*\text{hydro_height_reduction}_{h}*\text{hydro}_{h,p}} \quad \forall p \in \text{Periods}
\end{equation}

- Cyclic constraint: Height at the first period must be equal to height at the last period.

\begin{equation}
\text{height}_{pfirst} = \text{height}_{plast}  + \text{period_hours}_{pfirst}*\text{pumping}_{pfirst}/3000 -
\sum_{h \in \text{HydroUnits}}{\text{period_hours}_{pfirst}*\text{hydro_height_reduction}_{h}*\text{hydro}_{h,pfirst}}
\end{equation}

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [3]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

ModuleNotFoundError: No module named 'pandas'

## Input Data
We define all the input data of the model.

In [ ]:
# Parameters

ntypes = 3
nperiods = 5
maxstart0 = 5
hydrounits = 2

generators = [12, 10, 5]
period_hours = [6, 3, 6, 3, 6]
demand = [15000, 30000, 25000, 40000, 27000]
min_load = [850, 1250, 1500]
max_load = [2000, 1750, 4000]
base_cost = [1000, 2600, 3000]
per_mw_cost = [2, 1.3, 3]
startup_cost = [2000, 1000, 500]

hydro_load = [900, 1400]
hydro_cost = [90, 150]
hydro_height_reduction = [0.31, 0.47]
hydro_startup_cost = [1500, 1200]

## Model Deployment

We create a model and the variables. For each time period, we have: an integer decision variable to capture the number of active generators of each type (ngen), an integer variable to capture the number of generators of each type we must start (nstart), a continuous decision variable to capture the total power output for each generator type (output), a binary decision variable that indicates whether a hydro unit is active (hydro), a binary decision variable that indicates whether a hydro unit must be started (hydrstart), a continuous decision variable that captures the amount of enery used to replenish the reservoir (pumping), and a continuous decision variable that captures the height of the reservoir (height).

In [ ]:
model = gp.Model('PowerGeneration2')

ngen = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="ngen")
nstart = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="nstart")
output = model.addVars(ntypes, nperiods, vtype=GRB.CONTINUOUS, name="genoutput")

hydro = model.addVars(hydrounits, nperiods, vtype=GRB.BINARY, name="hydro")
hydrostart = model.addVars(hydrounits, nperiods, vtype=GRB.BINARY, name="hydrostart")
pumping = model.addVars(nperiods, vtype=GRB.CONTINUOUS, name="pumping")
height = model.addVars(nperiods, vtype=GRB.CONTINUOUS, name="height")

Next we insert the constraints:

The number of active generators can't exceed the number of generators:

In [ ]:
# Generator count

numgen = model.addConstrs(ngen[type, period] <= generators[type]
                         for type in range(ntypes) for period in range(nperiods))

Total power output for a thermal generator type depends on the number of generators of that type that are active.

In [ ]:
# Respect minimum and maximum output per generator type

min_output = model.addConstrs((output[type, period] >= min_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

max_output = model.addConstrs((output[type, period] <= max_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

Total generator output (thermal plus hydro) for each time period must meet predicted demand plus pumping.

In [ ]:
# Meet demand

meet_demand = model.addConstrs(gp.quicksum(output[type, period] for type in range(ntypes)) +
                               gp.quicksum(hydro_load[unit]*hydro[unit,period] for unit in range(hydrounits))
                               >= demand[period] + pumping[period]
                               for period in range(nperiods))

Maintain appropriate reservoir levels

In [ ]:
# Reservoir levels

reservoir = model.addConstrs(height[period] == height[period-1] + period_hours[period]*pumping[period]/3000 -
                             gp.quicksum(hydro_height_reduction[unit]*period_hours[period]*hydro[unit,period] for unit in range(hydrounits))
                             for period in range(1,nperiods))

# cyclic - height at end must equal height at beginning
reservoir0 = model.addConstr(height[0] == height[nperiods-1] + period_hours[0]*pumping[0]/3000 -
                             gp.quicksum(hydro_height_reduction[unit]*period_hours[0]*hydro[unit,0]
                             for unit in range(hydrounits)))

Selected generators must be able to cope with an excess of demand.

In [ ]:
# Provide sufficient reserve capacity

reserve = model.addConstrs(gp.quicksum(max_load[type]*ngen[type, period] for type in range(ntypes)) >=
                           1.15*demand[period] - sum(hydro_load)
                           for period in range(nperiods))

Connect the decision variables that capture active thermal generators with the decision variables that count startups.

In [ ]:
# Startup constraint

startup0 = model.addConstrs((ngen[type,0] <= maxstart0 + nstart[type,0])
                            for type in range(ntypes))

startup = model.addConstrs((ngen[type,period] <= ngen[type,period-1] + nstart[type,period])
                           for type in range(ntypes) for period in range(1,nperiods))

Connect hydro decision variables with hydro startup decision variables.

In [ ]:
# Hydro startup constraint

hydro_startup0 = model.addConstrs((hydro[unit,0] <= hydrostart[unit,0])
                                    for unit in range(hydrounits))

hydro_startup = model.addConstrs((hydro[unit,period] <= hydro[unit,period-1] + hydrostart[unit,period])
                           for unit in range(hydrounits) for period in range(1,nperiods))

Objective: minimize total cost.  Cost consists of five components: the cost for running active thermal generation units, the cost to generate power beyond the minimum for each unit, the cost to start up thermal generation units, the cost to operate hydro units, and the cost to start up hydro units.

In [ ]:
# Objective: minimize total cost

active = gp.quicksum(base_cost[type]*period_hours[period]*ngen[type,period]
                    for type in range(ntypes) for period in range(nperiods))

per_mw = gp.quicksum(per_mw_cost[type]*period_hours[period]*(output[type,period] - min_load[type]*ngen[type,period])
                       for type in range(ntypes) for period in range(nperiods))

startup_obj = gp.quicksum(startup_cost[type]*nstart[type,period]
                         for type in range(ntypes) for period in range(nperiods))

hydro_active = gp.quicksum(hydro_cost[unit]*period_hours[period]*hydro[unit,period]
                           for unit in range(hydrounits) for period in range(nperiods))

hydro_startup = gp.quicksum(hydro_startup_cost[unit]*hydrostart[unit,period]
                            for unit in range(hydrounits) for period in range(nperiods))

model.setObjective(active + per_mw + startup_obj + hydro_active + hydro_startup)

Next, we start the optimization and Gurobi finds the optimal solution.

In [ ]:
model.optimize()

---
## Analysis

The anticipated demand for electricity over the 24-hour time window can be met for a total cost of $\$1,000,630$, as compared with the $\$1,002,540$ that was required to meet the same demand without the hydro plants.  The detailed plan for each time period is as follows.

### Unit Commitments

The following table shows the number of thermal generators of each type that are active in each time period in the optimal solution:

In [ ]:
rows = ["Thermal" + str(t) for t in range(ntypes)]
units = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t in range(ntypes):
    for p in range(nperiods):
        units.loc["Thermal"+str(t), p] = ngen[t,p].x
units

The following shows the number of generators of each type that must be started in each time period to achieve this plan (recall that the model assumes that 5 are available at the beginning of the time horizon):

In [ ]:
rows = ["HydroA", "HydroB"]
hydrotable = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for p in range(nperiods):
    hydrotable.loc["HydroA", p] = int(hydro[0,p].x)
    hydrotable.loc["HydroB", p] = int(hydro[1,p].x)
hydrotable

The hydro plants are lightly used in the schedule - we only use plant B for the last two time periods.

The following shows the pumping that must be done in order to support this level of hydro activity

In [ ]:
pumptable = pd.DataFrame(columns=range(nperiods), index=["Pumping"], data=0.0)

for p in range(nperiods):
        pumptable.loc["Pumping", p] = pumping[p].x
pumptable

It is interesting to note that the plan simultaneously operates HydroB and pumps in time period 4.
While it might appear that costs could be reduced by operating the hydro plant at a lower load, in this model the hydro plants have fixed output.  If it makes economic sense to turn a hydro plant on, then we have to replenish the water drained, even if it means using some of the generated electricity to pump.

---
## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC